In [12]:
# Standard library imports
import os
import json
from glob import glob

# Third-party imports
import tifffile
import ants
import numpy as np

# ClearEx imports
from clearex.registration.common import export_tiff


In [13]:
# Find all files starting with "round1_" in the data directory (not round10, round11, etc.)
save_directory = "/archive/bioinformatics/Danuser_lab/Dean/dean/2025-11-30"
transform_directory = "/archive/bioinformatics/Danuser_lab/Dean/dean/2025-11-28"
data_directory = "/archive/bioinformatics/Danuser_lab/Dean/Seweryn/Manuscripts/Cyclic_Expansion/cyclic_manuscript_OPM_final/registration/cell4"

imaging_round = 1

# Load crop indices for round 1
crop_indices_path = os.path.join(
    transform_directory, f"fixed_crop_indices_{imaging_round}.json"
)
with open(crop_indices_path, 'r') as f:
    crop_indices = json.load(f)
z0, z1, y0, y1, x0, x1 = crop_indices

pattern = os.path.join(data_directory, "round1_*")
image_files = sorted(glob(pattern))

print(f"\nFound {len(image_files)} files matching 'round{imaging_round}_*':")
for file_to_load in image_files:
    print(f"  {os.path.basename(file_to_load)}")

    image_path = os.path.join(data_directory, file_to_load)
    image = tifffile.imread(image_path)

    # Crop the image.
    image = image[z0:z1, y0:y1, x0:x1]

    # Save data to save_directory
    file_to_save_to = os.path.join(save_directory, os.path.basename(file_to_load))
    tifffile.imwrite(file_to_save_to, image)


    file_to_save_to = os.path.join(save_directory, f"max_{os.path.basename(file_to_load)}")
    tifffile.imwrite(file_to_save_to, np.max(image, axis=0))




Found 4 files matching 'round1_*':
  round1_405_nuclei.tif
  round1_488_tubulin.tif
  round1_555_actin.tif
  round1_647_ER.tif


In [14]:
print("Starting transformations...")
fixed_image_path = os.path.join(save_directory, "round1_488_tubulin.tif")
fixed_image = tifffile.imread(fixed_image_path)
fixed_image = ants.from_numpy(fixed_image)

for imaging_round in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]:
    print("Processing round:", imaging_round)

    # Load crop indices for each round
    crop_indices_path = os.path.join(transform_directory, f"fixed_crop_indices_{imaging_round}.json")
    with open(crop_indices_path, 'r') as f:
        z0, z1, y0, y1, x0, x1 = json.load(f)

    # Transformation files
    linear_transform_path = os.path.join(transform_directory, f"linear_transform_{imaging_round}.mat")
    nonlinear_transform_path = os.path.join(transform_directory, f"nonlinear_warp_{imaging_round}.nii.gz")
    print("Loaded transforms for round", imaging_round)

    # Find all of the files for this round.
    pattern = os.path.join(data_directory, f"round{imaging_round}_*")
    image_files = sorted(glob(pattern))
    print(f"\nFound {len(image_files)} files matching 'round{imaging_round}_*':")

    # Iterate through each file.
    for file_to_load in image_files:
        print(f"  {os.path.basename(file_to_load)}")

        # Load the image.
        image_path = os.path.join(data_directory, file_to_load)
        image = tifffile.imread(image_path)

        # Crop the image.
        image = image[z0:z1, y0:y1, x0:x1]

        # Get min/max for proper scaling
        min_val = float(image.min())
        max_val = float(image.max())

        # Convert to ANTs image
        image = ants.from_numpy(image)

        transformed = ants.apply_transforms(
            fixed=fixed_image,
            moving=image,
            transformlist=[
                nonlinear_transform_path,
                linear_transform_path
            ],
            whichtoinvert=[False, False],
            interpolator="linear"
        )

        file_to_save_to = os.path.join(save_directory, os.path.basename(file_to_load))
        export_tiff(transformed, file_to_save_to, min_value=min_val, max_value=max_val)

        file_to_save_to = os.path.join(save_directory, f"max_{os.path.basename(file_to_load)}")
        export_tiff(transformed, file_to_save_to, min_value=min_val, max_value=max_val, max_intensity_project=True)





Starting transformations...
Processing round: 2
Loaded transforms for round 2

Found 3 files matching 'round2_*':
  round2_488_tubulin.tif
  round2_555_NUP.tif
  round2_647_lamin.tif
Processing round: 3
Loaded transforms for round 3

Found 3 files matching 'round3_*':
  round3_488_tubulin.tif
  round3_555_vinculin.tif
  round3_647_endosomes.tif
Processing round: 4
Loaded transforms for round 4

Found 3 files matching 'round4_*':
  round4_488_vimentin.tif
  round4_555_tubulin.tif
  round4_647_tom20.tif
Processing round: 5
Loaded transforms for round 5

Found 3 files matching 'round5_*':
  round5_488_cdc42.tif
  round5_555_tubulin.tif
  round5_647_lysosomes.tif
Processing round: 6
Loaded transforms for round 6

Found 3 files matching 'round6_*':
  round6_488_tubulin.tif
  round6_555_ctlc.tif
  round6_647_peroxisomes.tif
Processing round: 7
Loaded transforms for round 7

Found 2 files matching 'round7_*':
  round7_488_tubulin.tif
  round7_647_drp1.tif
Processing round: 8
Loaded transforms